<a href="https://colab.research.google.com/github/aderdouri/ql_web_app/blob/master/ql_notebooks/forwardrateagreement.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install QuantLib-Python

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 20.0/20.0 MB 25.3 MB/s eta 0:00:00


In [ ]:
import QuantLib as ql
import unittest
import math # For math.fabs

class ForwardRateAgreementTests(unittest.TestCase):

    def setUp(self):
        self.saved_eval_date = ql.Settings.instance().evaluationDate
        # Set a default evaluation date for the test, can be overridden
        # Using a date that ensures USDLibor fixings are valid
        ql.Settings.instance().evaluationDate = ql.Date(15, ql.May, 2020)

    def tearDown(self):
        ql.Settings.instance().evaluationDate = self.saved_eval_date

    def testConstructionWithoutACurveInitially(self): # Renamed to reflect the test's core idea
        """Testing forward rate agreement construction when curve is linked later."""
        print("Testing forward rate agreement construction...")

        today = ql.Settings.instance().evaluationDate

        # Set up the index
        curve_handle = ql.RelinkableYieldTermStructureHandle()
        index = ql.USDLibor(ql.Period(3, ql.Months), curve_handle)

        # Determine the settlement date for a FRA
        # index.fixingCalendar().advance(referenceDate, n, unit, convention=Following, endOfMonth=False)
        # index.fixingDays() is an integer
        settlement_date = index.fixingCalendar().advance(today, index.fixingDays(), ql.Days)

        # Set up quotes with no initial values (will be set later)
        # In Python, SimpleQuote defaults to Null<Real>() if no value given.
        quotes_py = [
            ql.SimpleQuote(),
            ql.SimpleQuote(),
            ql.SimpleQuote()
        ]
        quote_handles = [ql.QuoteHandle(q) for q in quotes_py]

        # Determine useIndexedFra based on QL version/compilation flags
        # This logic is from the C++ preprocessor directives.
        # In Python, we can check if ql.IndexedCoupon is available as an indicator,
        # or assume the typical behavior based on common QL builds.
        # If ql.FraRateHelper takes 'useIndexedCoupon' directly, that's simpler.
        # The Python FraRateHelper constructor doesn't seem to take 'useIndexedCoupon'.
        # It might default based on QL's build. For testing, we proceed assuming
        # the default behavior of FraRateHelper is consistent with one of the C++ paths.
        # Typically, modern QL would use indexed coupons if available.
        # Let's assume use_indexed_fra = True (which is default if QL_USE_INDEXED_COUPON is not defined or for older QL).
        # If QL_USE_INDEXED_COUPON is defined, it means standard coupons are used (useIndexedFra=false).
        # This is tricky to replicate perfectly without knowing the build.
        # For now, let's assume the default FraRateHelper behavior is what's intended for the test.
        # Python's FraRateHelper doesn't expose this bool directly.

        # Set up the curve helpers
        helpers = []
        # FraRateHelper(rate, periodToStart, iborIndex, pillar=ql.Pillar.LastRelevantDate, customPillarDate=ql.Date())
        helpers.append(ql.FraRateHelper(quote_handles[0], ql.Period(1, ql.Years), index, ql.Pillar.LastRelevantDate, ql.Date()))
        helpers.append(ql.FraRateHelper(quote_handles[1], ql.Period(2, ql.Years), index, ql.Pillar.LastRelevantDate, ql.Date()))
        helpers.append(ql.FraRateHelper(quote_handles[2], ql.Period(3, ql.Years), index, ql.Pillar.LastRelevantDate, ql.Date()))

        # Build the curve
        # PiecewiseYieldCurve<ForwardRate, Cubic>(referenceDate, helpers, dayCounter, jumps, jumpDates, accuracy, I())
        # In Python, the template args are part of the name: PiecewiseForwardRateCubicCurve
        # PiecewiseYieldCurve_ForwardRate_Cubic(...) is not standard.
        # It's usually ql.PiecewiseLogCubicDiscount, ql.PiecewiseLinearZero, etc.
        # For ForwardRate trait and Cubic interpolation:
        # ql.PiecewiseYieldCurve[ql.ForwardRate, ql.Cubic] in C++
        # Python binding is often specific, e.g., ql.PiecewiseLogCubicDiscount, ql.PiecewiseLinearZero.
        # If a direct 'PiecewiseForwardRateCubicCurve' is not available, we might need
        # to use a similar one (e.g., LogCubicDiscount) and acknowledge potential minor differences
        # or verify the exact Python binding name for this combination.
        # Common ones are Discount, Zero, ForwardRate traits and interpolators like Linear, LogLinear, Cubic, LogCubic.
        # Let's try with a common one, e.g., PiecewiseLogCubicDiscount, and see if it causes issues.
        # The key is that it's a Piecewise curve.
        # The specific choice of Trait/Interpolator impacts the curve shape.
        # If the test expects specific behavior from ForwardRate trait and Cubic interpolation,
        # we need the exact Python name.
        # A common combination for forward rates might be ql.PiecewiseLinearForward or ql.PiecewiseCubicZero.
        # Let's assume `ql.PiecewiseLogLinearForward` for now as an example, or a more generic `PiecewiseYieldCurve`.
        #
        # Looking at available Python classes: `ql.PiecewiseFlatForward`, `ql.PiecewiseLinearForward`
        # `ql.PiecewiseLogLinearDiscount`, `ql.PiecewiseLogCubicDiscount`
        # `ql.PiecewiseLinearZero`, `ql.PiecewiseSplineCubicDiscount` (which might be `PiecewiseCubicDiscount`)
        # Let's use `PiecewiseLinearForward` as it's a forward rate curve, though not cubic.
        # If cubic forward rate curve is essential, we need `PiecewiseCubicForward` or similar.
        #
        # Ah, the C++ `PiecewiseYieldCurve<ForwardRate, QuantLib::Cubic>` likely maps to
        # a Python class that embeds "ForwardRate" and "Cubic" in its name if directly wrapped.
        # Given `ql.PiecewiseLogCubicDiscount`, `ql.PiecewiseLinearZero`, etc.
        # A direct `PiecewiseForwardRateCubic` seems less common in standard Python bindings.
        # Let's use `ql.PiecewiseFlatForward` for simplicity in setting up a curve,
        # recognizing this is a deviation from "Cubic". The test seems more focused on FRA construction
        # and rate retrieval than the exact curve shape from a specific interpolator.
        # If the test fails due to this, we'd need the precise Python equivalent of `PiecewiseYieldCurve<ForwardRate, Cubic>`.
        # A safe bet might be `ql.PiecewiseLogCubicDiscount` and checking if `index.dayCounter()` is appropriate.

        # For `PiecewiseYieldCurve<ForwardRate, Cubic>` => A common Python binding is often `PiecewiseCubicZero`
        # or `PiecewiseCubicForward` if available. Let's try with `PiecewiseLinearForward` which is simpler.
        # The core of this test is about FRA construction and if it can pick up the rate later.

        # Actually, `PiecewiseYieldCurve[ForwardRate,LogLinear]` is often available as `PiecewiseLogLinearForwardRate`.
        # For Cubic, it would be `PiecewiseCubicForwardRate`. If not, `PiecewiseCubicZeroRate` is an alternative.
        # Let's use `PiecewiseLinearForward` to ensure it constructs.
        curve = ql.PiecewiseLinearForward(today, helpers, index.dayCounter())
        # curve = ql.PiecewiseLogCubicDiscount(today, helpers, index.dayCounter()) # Alternative

        curve_handle.linkTo(curve)

        # Set up the instrument to price
        # FRA constructor: (index, valueDate, maturityDate (optional), position, strike, notional, yieldCurveHandle)
        # If maturityDate is not given, it's inferred from the index's tenor.

        # Case 1: FRA without explicit maturity date (inferred from index tenor)
        # valueDate = settlementDate + Period(12, Months)
        # The FRA period starts at valueDate and ends valueDate + index.tenor()
        value_date_1 = settlement_date + ql.Period(12, ql.Months)

        fra1 = ql.ForwardRateAgreement(index,
                                       value_date_1,
                                       ql.Position.Long,
                                       0.0, # strike
                                       1.0, # notional
                                       curve_handle) # Pass the curve handle

        # Finally, set values in the quotes to trigger curve recalculation
        quotes_py[0].setValue(0.01)
        quotes_py[1].setValue(0.02)
        quotes_py[2].setValue(0.03)

        # Get the forward rate from the FRA
        rate1 = fra1.forwardRate()
        expected_rate1 = 0.01 # Based on the C++ test's expectation with the first quote

        # The expected rate of 0.01 implies that the 1Y FRA (starting in 1Y) rate is directly
        # taken from the 1Y helper. This would be true if the curve is flat up to that point
        # or if the interpolation gives this result. With PiecewiseLinearForward and a 1Y helper,
        # the forward rate for a period starting at 1Y matching the helper's tenor should be close to the helper's quote.
        # The FRA is for 3M (index tenor) starting in 12M. So, it's the 12M x 15M FRA.
        # The first helper is for a 1Y period. This is likely a simplification in the C++ test's expectation.
        # A 1Y FRA helper typically means a FRA covering the period from `today + fixingDays` to `today + fixingDays + 1Y`.
        # This is different from a FRA starting in 1Y.
        # Let's re-check the FraRateHelper: `Period(1, Years)` is `periodToStart`.
        # So, helper[0] is for a FRA starting in 1Y (e.g., 1Y vs 1Y3M).
        # If `value_date_1` aligns with the start of this helper, then the rate should be 0.01.
        # `value_date_1` is `settlementDate + 12M`. `settlementDate` is `today + fixingDays`.
        # If `fixingDays` is small (e.g., 2d), then `value_date_1` is roughly `today + 12M`.
        # This aligns with the first helper.

        self.assertAlmostEqual(rate1, expected_rate1, delta=1e-6,
                               msg=f"FRA rate (inferred maturity) failed: got {rate1}, expected {expected_rate1}")

        # Case 2: FRA with explicit maturity date
        value_date_2 = settlement_date + ql.Period(12, ql.Months)
        maturity_date_2 = settlement_date + ql.Period(15, ql.Months) # Explicit 3M tenor

        fra2 = ql.ForwardRateAgreement(index,
                                       value_date_2,
                                       maturity_date_2, # Explicit maturity date
                                       ql.Position.Long,
                                       0.0,  # strike
                                       1.0,  # notional
                                       curve_handle)

        rate2 = fra2.forwardRate()
        expected_rate2 = 0.01 # Same expectation
        self.assertAlmostEqual(rate2, expected_rate2, delta=1e-6,
                               msg=f"FRA rate (explicit maturity) failed: got {rate2}, expected {expected_rate2}")


if __name__ == '__main__':
    print("C++ test suite: ForwardRateAgreementTests")
    print("Python QuantLib version: " + ql.__version__)
    print("-------------------------------------------------------")
    suite = unittest.TestSuite()
    suite.addTest(unittest.makeSuite(ForwardRateAgreementTests))
    unittest.TextTestRunner(verbosity=2).run(suite)